In [1]:
from mpmath import mp, mpf, findroot, bessely
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

In [44]:
from sage.all import *

R = LaurentSeriesRing(RealField(300), 'x', default_prec=300)
x = R.gen()
N = 16  # Number of terms (adjust as needed)
gamma = RealField(300)(euler_gamma)
d2 = RealField(300)(2)
pi = RealField(300).pi()
log2 = RealField(300)(2).log()

def j_series(n, x):
    return sum([(-1)**m * (x/2)**(ZZ(n) + ZZ(2)*ZZ(m)) / (ZZ(m).factorial() * (ZZ(m) + ZZ(n)).factorial()) for m in range(N)])

J1_series = j_series(1, x)

def harmony(m):
    return sum(RealField(300)(1)/RealField(300)(k) for k in range(1, m+1))

def z_series(x):
    return sum([(-1)**m * (x)**(ZZ(2)*ZZ(m)) / (ZZ(2)**(2*m) * ZZ(m).factorial() * (ZZ(m) - ZZ(1)).factorial()) * (harmony(m) + harmony(m - 1)) for m in range(1, N)])

W1 = d2/pi * J1_series
Z1 = -(d2/(x*pi) * z_series(x) + d2/pi * gamma * J1_series(x) - d2/pi * log2 * J1_series(x))

def y1_full(x):
    return d2/pi * (J1_series(x) * x.log() - 1/x * ( 1 - z_series(x)) + (gamma - log2) * J1_series(x))

def y1_full_expanded(x):
    return d2/pi * J1_series(x) * x.log() - d2/(x*pi) + d2/(x*pi) * z_series(x) + d2/pi * gamma * J1_series(x) - d2/pi * log2 * J1_series(x)

# print("W0(x) =", W0)
# print("Z0(x) =", Z0)

terms = 30

from mpmath import mp, bessely, taylor

def Y1_approx(val):
    # Substitute numeric value and convert to high precision real
    w = W1.truncate(terms)(val)
    z = Z1.truncate(terms)(val)
    return RealField(107)(w) * RealField(107)(val).log() - RealField(107)(z) - RealField(107)(2)/(pi * val)

mp.prec = 120

print("W = ", RealField(64)(W1.truncate(terms)(0.89357696627916749)))
print("Z = ", RealField(64)(Z1.truncate(terms)(0.89357696627916749)))
print("LOG = ", RealField(110)(0.89357696627916749).log())
print_double_double("", RealField(110)(2).log())
print("Y1 approx = ", Y1_approx(RealField(100)(0.89357696627916749)))
print("Y1 full = ", y1_full(RealField(100)(0.89357696627916749)))
print("Y1 full expanded = ", y1_full_expanded(RealField(100)(0.89357696627916749)))
print("Y1 = ", bessely(1, 0.89357696627916749))

print(W1.truncate(10))

def print_series(z, N):
    for i in range(N):
        coeff = z[i]
        if coeff != 0:
            print(double_to_hex(RealField(300)(coeff)) + ",")
            # print_double_double("", RealField(300)(coeff))

print_series(W1, 20)

print("2/PI", double_to_hex(d2/pi))

W =  0.256973952516943105
Z =  0.138065597292926445
LOG =  -0.11252280788079415479644853102767
(0x3c7abc9e3b39803f, 0x3fe62e42fefa39ef),
Y1 approx =  -0.87942080249719479722972442056
Y1 full =  -0.87942080249719479722972442056
Y1 full expanded =  -0.87942080249719479722972442056
Y1 =  -0.87942080249719479722972442056326109
0.318309886183790671537767526745028724068919291480912897495334688117793595268453070180227606*x + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 - 0.0397887357729738339422209408431285905086149114351141121869168360147241994085566337725284507*x^3 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.00165786399054057641425920586846369127119228797646308800778820150061350830868985974052201878*x^5 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 - 0.000034538833136262008630400122259660234816505999509647666828920864596114756431

In [49]:
R120 = RealField(120)

zeros = []

mp.prec = 150

step = mpf("0.1")
epsilon = mpf("1e-35")
x = mpf("1.25")

previous_zero = R120(0)
y1_zeros = []

while x < mpf("52.0"):
    f1 = bessely(1, x)
    f2 = bessely(1, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: bessely(1, t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        previous_zero = zero
        y1_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

y1_extrema = []

x = mpf("1.25")
while x < mpf("52.0"):
    d1 = mp.diff(lambda t: bessely(1, t), x)
    d2 = mp.diff(lambda t: bessely(1, t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: bessely(1, u), t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        y1_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(y1_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of Y1(x):")
for e in y1_extrema:
    print(f"nExtrema: {e}")

y1_zeros.extend(y1_extrema)

y1_zeros = sorted(y1_zeros)

# Print results
for i, z in enumerate(y1_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

print(f"pub(crate) static Y1_ZEROS: [(u64, u64); {len(y1_zeros)}] = [")
print(f"(0x0, 0x0),")
for z in y1_zeros:
    k = split_double_double(z)
    hi = double_to_hex(k[1])
    lo = double_to_hex(k[0])
    print(f"({lo}, {hi}),")
    
print("];")

Zero 1: x ≈ 2.1971413260310170351490335626989662730530183
Zero 2: x ≈ 5.4296810407941351327720051908525841965837575
Zero 3: x ≈ 8.5960058683311689264296061801639678511029216
Zero 4: x ≈ 11.749154830839881243399421939922350714301166
Zero 5: x ≈ 14.897442128336725378844819156429870879807151
Zero 6: x ≈ 18.043402276727855564304555507889508902163088
Zero 7: x ≈ 21.188068934142213016142481528685423196935025
Zero 8: x ≈ 24.331942571356912035992944051850129651414333
Zero 9: x ≈ 27.475294980449223512212285525410668235700897
Zero 10: x ≈ 30.61828649164111471576162569644744831027794
Zero 11: x ≈ 33.761017796109325692471759911249650993879821
Zero 12: x ≈ 36.903555316142950053588159309844303659060545
Zero 13: x ≈ 40.045944640266876089064214537312936172020628
Zero 14: x ≈ 43.188218097393211268477533200127597207456358
Zero 15: x ≈ 46.330399250701686587724463565243621670014636
Zero 16: x ≈ 49.472505679924095824128003887609267273294894
Extrema (peaks/valleys) of Y1(x):
nExtrema: 3.68302285658517769989

In [60]:
mp.prec = 60
terms = 29
print(f"pub(crate) static Y1F_COEFFS: [[u64; {terms}]; {len(y1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, n):
    print("[")
    for i in range(0, n):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(y1_zeros)):
    k_range = y1_zeros[i]
    range_diff = k_range - prev_zero

    x0 = mp.mpf(k_range)
    from mpmath import mp, bessely, taylor
    poly = taylor(lambda val: bessely(1, val), x0, terms)
    # print(poly)
    print_taylor_coeffsf(poly, terms)
    prev_zero = y1_zeros[i]

print("];")

pub(crate) static Y1F_COEFFS: [[u64; 29]; 32] = [
[
0x0000000000000000,
0x3fe0aa48442f014b,
0xbfbe56f82217b8f1,
0xbfa0d2af4e932386,
0xbf73a6dec37290aa,
0x3f7e671c7d1196fd,
0xbf65429dc5a45612,
0x3f517ab4afac0735,
0xbf40b2d877d5a29c,
0x3f2eea7bbb907646,
0xbf1c3fae3b0ad706,
0x3f09d174c9ccecd9,
0xbef7956ad3fe9783,
0x3ee5865d11ebb14d,
0xbed3a27f1fd4ac6e,
0x3ec1e742c1cf6acb,
0xbeb051d134cb0a72,
0x3e9dbea846629267,
0xbe8b1a1682f5eb1f,
0x3e78b0b08d8ccddf,
0xbe667d8a7cc18981,
0x3e547c0e174bc402,
0xbe42a7fb50d94c09,
0x3e30fd661ade45f0,
0xbe1ef1655b283bca,
0x3e0c2d21fe733c5f,
0xbdf9a7efb711afac,
0x3de75c3fc5194fe2,
0xbdd544fd359e8d1a,
],
[
0x3fdaabb4011ed330,
0x0000000000000000,
0xbfc8b45babe797ba,
0x3f8e147099a6f0ea,
0x3f88c5af1eeb2d6b,
0xbf4133fa47da52e2,
0xbf3bf8af93ff0b8c,
0x3f021d64be62ccf9,
0x3eb44d2ce67b2d18,
0x3eb14c3ab2ad79cc,
0xbe9b8eee52e1ce01,
0x3e7a85f1a878746a,
0xbe5bf86b3c89849b,
0x3e3f01920a1ecb6b,
0xbe20fac544c5a674,
0x3e02818841d3a289,
0xbde42b5ceddd3872,
0x3dc5f9472a38633d,
0xb